# Interest Rate Prediction

### Setup and Loading the Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load dataset
url = "https://fred.stlouisfed.org/series/FEDFUNDS"
data = pd.read_csv("FEDFUNDS.csv")

# Data inspection
print(data.head())
print(data.info())

         DATE  FEDFUNDS
0  1954-07-01      0.80
1  1954-08-01      1.22
2  1954-09-01      1.07
3  1954-10-01      0.85
4  1954-11-01      0.83
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 844 entries, 0 to 843
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DATE      844 non-null    object 
 1   FEDFUNDS  844 non-null    float64
dtypes: float64(1), object(1)
memory usage: 13.3+ KB
None


### Data Preprocessing and Feature Engineering

In [ ]:
# Data preprocessing
# Convert date to datetime format and filter for relevant data
data['DATE'] = pd.to_datetime(data['DATE'])
data = data.set_index('DATE')

# Remove rows with missing values
data = data.dropna()

# Feature engineering
# Use lagged interest rates as features
data['Lag_1'] = data['FEDFUNDS'].shift(1)
data['Lag_2'] = data['FEDFUNDS'].shift(2)
data['Lag_3'] = data['FEDFUNDS'].shift(3)
data = data.dropna()

# Define input features (X) and target variable (y)
X = data[['Lag_1', 'Lag_2', 'Lag_3']]
y = data['FEDFUNDS']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Feature Standardization

In [4]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Building and Compiling the Initial Model

In [5]:
# Build the initial model
model = Sequential([
    Dense(64, activation='relu', input_dim=X_train_scaled.shape[1]),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

c:\Users\laura\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### TO DO: Train the model, evaluate performance, visualize results, and make modifications based on results (repeat)